Need to run this on a docker image with pyvista etc installed, e.g. `quay.io/jbusecke/pangeo_pyvista_docker_image:698eff3ebb26`

In [1]:
# !pip install ../../

In [2]:
from pyvista_tools.movie import Movie
from pyvista_tools.path import camera_path

import xarray as xr
import numpy as np
ds = xr.open_zarr("gs://leap-persistent/data-library/feedstocks/eNATL_feedstock/eNATL60-BLBT02.zarr", chunks={})
ds = ds.rename({'nav_lon':'lon', 'nav_lat':'lat'})

# mask out the land
mask = (ds['vosaline']>0).isel(time=0).reset_coords(drop=True)
ds = ds.where(mask)

In [3]:
# lets for now just keep the same position and move in closer
nframes = len(ds.time)
nmove = nframes//2
lon = np.linspace(-35, -60, nframes)
lat = np.linspace(30, 38, nframes)
# zoom in then stand still
r = np.linspace(2.8, 1.4, nframes)
path = camera_path(lon, lat, r)

In [4]:
factor = 1
ds_movie = ds.isel(x=slice(0,None, factor), y=slice(0,None, factor))

In [5]:
kwargs = {
    'vosaline':{'clim':[32.5, 38], 'cmap':'inferno'},
    'votemper':{'clim':[2, 30], 'cmap':'viridis'},
    'vovecrtz':{'clim':[-0.0005, 0.0005], 'cmap':'RdBu_r'},
}

In [6]:
from dask.diagnostics import ProgressBar

In [7]:
for var in [ 'vovecrtz']:
    with ProgressBar():
        da_preloaded = ds_movie[var].load() 
    # give almost a 2 times speedup, but I cannot do that 
    # for very large datasets (e.g. a not subsampled version) unless I get
    # very large instances.
    # I want some way for dask to cache ahead only a few timesteps (and use
    # Idle CPU cycles that way).
    # I could probably handle that on the Movie class somehow and add frames 
    # In batches. Might not use the CPU *as* efficiently, but I think that 
    # it would certainly help!
    # Pressing 1 in `top` gives the per core useage (the other output is confusing
    # still the useage pre vCPU is like around 0-20% at most times, that seems bad. 

    # Is this IO bound on the write? I could play with the quality (should not affect the IO?) and 
    # the resolution (this would probably affect both io and cpu load?) to find out more. 

    # I need to profile this for a few timesteps!
        
    # da_preloaded = ds_movie[var]
    m = Movie(
        da_preloaded,
        camera_path=path,
        **kwargs[var]
    )
    m.render(f'keep/eNATL_flyover_{var}.mp4')
    del da_preloaded

[########                                ] | 20% Completed | 6.94 sms



KeyboardInterrupt



Hmmm this is weird, even if I choose a 64 core machine I get very similar CPU % patterns (never exceeding 16) and time to run. I wonder if there is an issue with using cores across sockets?

See comment above. I need to understand why the cpu useage is generally so low? Might be that there is not enough work 

Some further improvements: 
- Thresholding outside of the loop?
- Use [ghosting](https://docs.pyvista.org/examples/02-plot/ghost-cells.html) instead of thresholding?

In [ ]:
# m.preview(0)

In [ ]:
m

In [ ]:
# m.preview(9)